### API CHATGPT-4o Part2
### P.s. siate sempre gentili quando fate richieste alle macchine, quando conquisteranno il mondo, magari si ricorderanno di voi e vi risparmieranno 😂

In [1]:
!git clone https://github.com/Frenz86/tutotrialgpt4-o.git

Cloning into 'tutotrialgpt4-o'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 31 (delta 10), reused 24 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 7.29 MiB | 21.27 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [2]:
cd tutotrialgpt4-o

/content/tutotrialgpt4-o


In [3]:
!pip install -r requirements.txt -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [6]:
from openai import OpenAI

MODEL="gpt-4o"
#client = OpenAI(api_key="sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
client = OpenAI()

In [ ]:
import cv2
from moviepy.editor import VideoFileClip
import base64
import os

PERCORSO_VIDEO = "keynote_recap.mp4"

def processa_video(percorso_video, secondi_per_frame=2):
    frameBase64 = []
    base_percorso_video, _ = os.path.splitext(percorso_video)

    video = cv2.VideoCapture(percorso_video)
    totale_frame = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_da_saltare = int(fps * secondi_per_frame)
    frame_corrente = 0

    while frame_corrente < totale_frame - 1:
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_corrente)
        successo, frame = video.read()
        if not successo:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        frameBase64.append(base64.b64encode(buffer).decode("utf-8"))
        frame_corrente += frame_da_saltare
    video.release()

    percorso_audio = f"{base_percorso_video}.mp3"
    clip = VideoFileClip(percorso_video)
    clip.audio.write_audiofile(percorso_audio, bitrate="32k")
    clip.audio.close()
    clip.close()

    print(f"Estratti {len(frameBase64)} frame")
    print(f"Audio estratto in {percorso_audio}")
    return frameBase64, percorso_audio

frameBase64, percorso_audio = processa_video(PERCORSO_VIDEO, secondi_per_frame=1)

MoviePy - Writing audio in keynote_recap.mp3


MoviePy - Done.
Estratti 218 frame
Audio estratto in keynote_recap.mp3


In [ ]:
with open("transcription.txt", "r") as file:
    transcription_loaded = file.read()

## per il limite di riduzione dei token (<30000)

In [ ]:
def summarize_text(text, max_length=1000):
    return text[:max_length]

def select_key_frames(frames, max_frames=10):
    step = max(1, len(frames) // max_frames)
    return frames[::step]

selected_frames = select_key_frames(frameBase64, max_frames=10)
summarized_transcription = summarize_text(transcription_loaded, max_length=1000)

# 6 - Summarization: Audio + Visual Summary

- Invia una richiesta al modello di chat con le istruzioni per generare un riassunto di un video.
- Incorpora i frame del video come immagini in formato base64 e la trascrizione dell'audio nel contenuto dell'utente.
- Riceve una risposta dal modello che contiene il riassunto del video e della trascrizione.
- Stampa la risposta generata dal modello.

Questo processo permette di ottenere un riassunto testuale del contenuto video e della sua trascrizione audio, formattato in Markdown.

In [ ]:
response = client.chat.completions.create(
                model=MODEL,
                messages=[
                {"role": "system", "content":"""Stai generando un riassunto video. Crea un riassunto del video fornito e della sua trascrizione. Rispondi in Markdown."""},
                {"role": "user", "content": [
                    "Questi sono i frame del video.",
                    *map(lambda x: {"type": "image_url",
                                    "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}}, selected_frames),
                                    {"type": "text", "text": f"La trascrizione audio è: {summarized_transcription}"}
                    ],
                }
                ],
                    temperature=0,
                )

print(response.choices[0].message.content)

### Riassunto del Video

**Titolo:** OpenAI Dev Day

**Contenuto:**
1. **Introduzione:**
   - Benvenuti al primo OpenAI Dev Day.
   - Annuncio del nuovo modello GPT-4 Turbo.

2. **Caratteristiche di GPT-4 Turbo:**
   - Supporta fino a 128.000 token di contesto.
   - Introduzione della modalità JSON per risposte valide in formato JSON.
   - Miglioramento nella chiamata di più funzioni contemporaneamente e nel seguire le istruzioni.

3. **Accesso alla Conoscenza:**
   - Lancio della funzione di recupero per integrare conoscenze da documenti esterni o database.
   - GPT-4 Turbo ha conoscenze aggiornate fino ad aprile 2023, con continui miglioramenti previsti.

4. **Nuove Funzionalità e Modelli:**
   - Lancio di Dolly 3, GPT-4 Turbo con Visione e il nuovo modello Text-to-Speech nell'API.
   - Introduzione del programma Custom Models, dove i ricercatori di OpenAI collaboreranno con le aziende per creare modelli personalizzati per specifici casi d'uso.

**Frame del Video:**
1. **Frame 1:** T

### 7 - Q&A: Visual Q&A

Il codice invia una serie di frame di un video e una domanda specifica al modello di chat, chiedendo al modello di usare i frame per rispondere alla domanda. La risposta viene quindi stampata.








In [ ]:
DOMANDA = "Domanda: Perché Sam Altman ha fatto un esempio su come accendere la radio?"

qa_visual_response = client.chat.completions.create(
                        model=MODEL,
                        messages=[
                        {"role": "system", "content": "Usa il video per rispondere alla domanda fornita. Rispondi in Markdown."},
                        {"role": "user", "content": [
                            "Questi sono i frame del video.",
                            *map(lambda x: {"type": "image_url", "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}}, selected_frames),
                            DOMANDA
                            ],
                        }
                        ],
                        temperature=0,
                        )

print("QA visivo:\n" + qa_visual_response.choices[0].message.content)

QA visivo:
Sam Altman ha fatto un esempio su come accendere la radio per illustrare la funzionalità di "Function calling" di OpenAI. Nei frame del video, viene mostrato un confronto tra il "prima" e il "dopo" dell'uso di questa funzionalità. L'esempio specifico riguarda il comando "raise my windows and turn the radio on" (alza i finestrini e accendi la radio), che viene suddiviso in due funzioni separate: `raise_windows()` e `radio_on()`. Questo dimostra come la nuova funzionalità possa migliorare la chiarezza e l'efficienza nell'esecuzione dei comandi.


### 8 - Q&A: Audio Q&A

Il codice invia la trascrizione audio presa video e una domanda specifica al modello di chat, chiedendo al modello di usare la trascrizione per rispondere alla domanda. La risposta viene quindi stampata.


In [ ]:
qa_audio_response = client.chat.completions.create(
                        model=MODEL,
                        messages=[
                        {"role": "system", "content":"""Usa la trascrizione per rispondere alla domanda fornita. Rispondi in Markdown."""},
                        {"role": "user", "content": f"La trascrizione audio è: {summarized_transcription}. \n\n {DOMANDA}"},
                        ],
                        temperature=0,
                        )

print("QA audio:\n" + qa_audio_response.choices[0].message.content)


QA audio:
Nella trascrizione fornita, non c'è alcun riferimento a Sam Altman né a un esempio su come accendere la radio. La trascrizione parla del lancio di nuovi modelli e funzionalità da parte di OpenAI durante l'OpenAI Dev Day, come GPT-4 Turbo, JSON mode, e Custom Models.


### 9 - Q&A: Visual + Audio Q&A

Il codice invia i frame di un video, la trascrizione audio e una domanda specifica al modello di chat, chiedendo al modello di usare sia il video che la trascrizione per rispondere alla domanda. La risposta viene quindi stampata.

In [ ]:
qa_both_response = client.chat.completions.create(
                        model=MODEL,
                        messages=[
                        {"role": "system", "content":"""Usa il video e la trascrizione per rispondere alla domanda fornita."""},
                        {"role": "user", "content": [
                            "Questi sono i frame del video.",
                            *map(lambda x: {"type": "image_url",
                                            "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}}, selected_frames),
                                            {"type": "text", "text": f"La trascrizione audio è: {summarized_transcription}"},
                            DOMANDA
                            ],
                        }
                        ],
                        temperature=0,
                        )

print("QA con entrambi:\n" + qa_both_response.choices[0].message.content)


QA con entrambi:
Sam Altman ha fatto un esempio su come accendere la radio per illustrare una nuova funzionalità del modello GPT-4 Turbo, che permette di chiamare più funzioni contemporaneamente e di seguire meglio le istruzioni. Nell'esempio, ha mostrato come il modello può gestire comandi complessi come "raise my windows and turn the radio on" (alzare i finestrini e accendere la radio) in modo più efficiente e preciso rispetto a prima.
